In [ ]:
import numpy as np
import xarray as xr
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import sys
import tripyview as tpv
import xgcm
import eddytools as et
import math
from cmocean import cm as cmo
import matplotlib.path as mpath
import cartopy.crs as ccrs
import pyfesom2 as pf
import seawater as sw
import matplotlib.tri as mtri

In [ ]:
figpath='/PATH/TO/OUTPUT/'
datapath='/PATH/TO/DATA/'
meshpath='/PATH/TO/MESH/'

In [ ]:
def simple_plot(dpi=300,
                ygridlocs=[-80,-75,-70,-65,-60],
                figsize=(6,6),
                box=[-180,180,-80,-60],
                cols=1,
                rows=1,
               ):
    
    fig, axes = plt.subplots(
                rows,cols,
                subplot_kw=dict(projection=ccrs.SouthPolarStereo()),
                constrained_layout=True,
                figsize=figsize,
                dpi=dpi,
                # facecolor='lightgrey',
            )
    if isinstance(axes, np.ndarray):
        axesflat = axes.flatten()
    else:
        axesflat = [axesflat]
    for ax in axesflat:
        ax.set_extent(box, crs=ccrs.PlateCarree())
        theta = np.linspace(0, 2*np.pi, 100)
        center, radius = [0.5, 0.505], 0.5
        verts = np.vstack([np.sin(theta), np.cos(theta)]).T
        circle = mpath.Path(verts * radius + center)

        ax.set_boundary(circle, transform=ax.transAxes)
        ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=False, linewidth=0.5, \
                xlocs=range(-180,171,30), ylocs=[], \
                color='gray', alpha=0.5, linestyle='--', zorder=10)
        # Draw concentric circles (but hide labels) for the parallels of the latitude
        ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=False, linewidth=0.5, \
                        xlocs=[], ylocs=ygridlocs, \
                        color='grey', alpha=0.5, linestyle='--', zorder=10)
        
    return fig, axes

In [ ]:
rm = cmo.tools.crop_by_percent(cmo.amp, 20, which='max', N=None)
rbm = cmo.tools.crop_by_percent(cmo.balance, 20, which='both', N=None)

In [ ]:
mesh=tpv.load_mesh_fesom2(meshpath, do_rot='None', focus=0, do_info=False, do_pickle=False,
                          do_earea=False, do_narea=False, do_eresol=[False,'mean'], do_nresol=[False,'eresol'])


In [ ]:
so3=pf.load_mesh(meshpath)

In [ ]:
#interpolation details
model_lons=so3.x2
model_lats=so3.y2

elements=so3.elem.astype('int32')

d = model_lons[elements].max(axis=1) - model_lons[elements].min(axis=1)
no_cyclic_elem = np.argwhere(d < 100).ravel()

triang = mtri.Triangulation(model_lons, model_lats, elements[no_cyclic_elem])
tri = triang.get_trifinder()

xint,yint=np.meshgrid(np.arange(-180,180.1,0.1),np.arange(-80,-58.9,0.1))

def interp(data):
    return mtri.LinearTriInterpolator(triang, data,trifinder=tri)(xint,yint)

In [ ]:
#streamfunction from tripyview - https://github.com/FESOM/tripyview/blob/main/templates_notebooks/template_transp_hbstreamf.ipynb
streamf50=xr.open_dataset(datapath+'hbstreamf_1951-1956.nc')
streamf90=xr.open_dataset(datapath+'hbstreamf_2091-2096.nc')

In [ ]:
#Surface wind. Forced with CMIP6 data. Paths for Levante - DKRZ.
c6uhistpath='/pool/data/CMIP6/data/CMIP/AWI/AWI-CM-1-1-MR/historical/r1i1p1f1/Amon/uas/gn/v20200511/'
c6vhistpath='/pool/data/CMIP6/data/CMIP/AWI/AWI-CM-1-1-MR/historical/r1i1p1f1/Amon/vas/gn/v20200511/'
c6uprojpath='/pool/data/CMIP6/data/ScenarioMIP/AWI/AWI-CM-1-1-MR/ssp370/r1i1p1f1/Amon/uas/gn/v20190529/'
c6vprojpath='/pool/data/CMIP6/data/ScenarioMIP/AWI/AWI-CM-1-1-MR/ssp370/r1i1p1f1/Amon/vas/gn/v20190529/'

u50files=[]
v50files=[]
u90files=[]
v90files=[]

for i in np.arange(6):
    u50files.append(c6uhistpath+'uas_Amon_AWI-CM-1-1-MR_historical_r1i1p1f1_gn_'+str(1951+i)+'01-'+str(1951+i)+'12.nc')
    v50files.append(c6vhistpath+'vas_Amon_AWI-CM-1-1-MR_historical_r1i1p1f1_gn_'+str(1951+i)+'01-'+str(1951+i)+'12.nc')

    u90files.append(c6uprojpath+'uas_Amon_AWI-CM-1-1-MR_ssp370_r1i1p1f1_gn_'+str(2091+i)+'01-'+str(2091+i)+'12.nc')
    v90files.append(c6vprojpath+'vas_Amon_AWI-CM-1-1-MR_ssp370_r1i1p1f1_gn_'+str(2091+i)+'01-'+str(2091+i)+'12.nc')
uw50=xr.open_mfdataset(u50files).mean(dim='time').compute()
uw90=xr.open_mfdataset(u90files).mean(dim='time').compute()
vw50=xr.open_mfdataset(v50files).mean(dim='time').compute()
vw90=xr.open_mfdataset(v90files).mean(dim='time').compute()

In [ ]:
#density calcs

In [ ]:
#reduce temp/salt data

In [ ]:
%%writefile ts_calcs.sh
module load cdo

datapath=/PATH/TO/DATA

cdo ymonmean -copy [ -sellevidx,19 $datapath/temp.fesom.1951.nc \
-sellevidx,19 $datapath/temp.fesom.1952.nc -sellevidx,19 $datapath/temp.fesom.1953.nc \
-sellevidx,19 $datapath/temp.fesom.1954.nc -sellevidx,19 $datapath/temp.fesom.1955.nc \
-sellevidx,19 $datapath/temp.fesom.1956.nc ] $datapath/temp_ymonmean_95m_1951-1956.nc &

cdo ymonmean -copy [ -sellevidx,19 $datapath/salt.fesom.1951.nc \
-sellevidx,19 $datapath/salt.fesom.1952.nc -sellevidx,19 $datapath/salt.fesom.1953.nc \
-sellevidx,19 $datapath/salt.fesom.1954.nc -sellevidx,19 $datapath/salt.fesom.1955.nc \
-sellevidx,19 $datapath/salt.fesom.1956.nc ] $datapath/salt_ymonmean_95m_1951-1956.nc &

cdo ymonmean -copy [ -sellevidx,19 $datapath/temp.fesom.2091.nc \
-sellevidx,19 $datapath/temp.fesom.2092.nc -sellevidx,19 $datapath/temp.fesom.2093.nc \
-sellevidx,19 $datapath/temp.fesom.2094.nc -sellevidx,19 $datapath/temp.fesom.2095.nc \
-sellevidx,19 $datapath/temp.fesom.2096.nc ] $datapath/temp_ymonmean_95m_2091-2096.nc &

cdo ymonmean -copy [ -sellevidx,19 $datapath/salt.fesom.2091.nc \
-sellevidx,19 $datapath/salt.fesom.2092.nc -sellevidx,19 $datapath/salt.fesom.2093.nc \
-sellevidx,19 $datapath/salt.fesom.2094.nc -sellevidx,19 $datapath/salt.fesom.2095.nc \
-sellevidx,19 $datapath/salt.fesom.2096.nc ] $datapath/salt_ymonmean_95m_2091-2096.nc &

In [ ]:
!bash ts_calcs.sh

In [ ]:
temp50=xr.open_dataset(datapath+'temp_ymonmean_95m_1951-1956.nc').isel(nz1=0)
salt50=xr.open_dataset(datapath+'salt_ymonmean_95m_1951-1956.nc').isel(nz1=0)
temp90=xr.open_dataset(datapath+'temp_ymonmean_95m_2091-2096.nc').isel(nz1=0)
salt90=xr.open_dataset(datapath+'salt_ymonmean_95m_2091-2096.nc').isel(nz1=0)

In [ ]:
#middle of level
pressures=sw.eos80.pres([97.5],so3.y2)

In [ ]:
dens50=np.mean(sw.eos80.pden(salt50.salt.values,temp50.temp.values,pressures.reshape([1,*pressures.shape]),[0]),axis=0)
dens90=np.mean(sw.eos80.pden(salt90.salt.values,temp90.temp.values,pressures.reshape([1,*pressures.shape]),[0]),axis=0)

In [ ]:
dens90_int=interp(dens90)
dens50_int=interp(dens50)

In [ ]:
#produce mke with cdo from notebook

In [ ]:
%%writefile mke_calcs.sh
module load cdo

datapath=/PATH/TO/DATA

cdo timmean -mulc,0.5 -add [ -sqr [ -copy [ -sellevidx,19 $datapath/unod.fesom.1951.nc \
-sellevidx,19 $datapath/unod.fesom.1952.nc -sellevidx,19 $datapath/unod.fesom.1953.nc \
-sellevidx,19 $datapath/unod.fesom.1954.nc -sellevidx,19 $datapath/unod.fesom.1955.nc \
-sellevidx,19 $datapath/unod.fesom.1956.nc ] ] \
-sqr [ -copy [ -sellevidx,19 $datapath/vnod.fesom.1951.nc \
-sellevidx,19 $datapath/vnod.fesom.1952.nc -sellevidx,19 $datapath/vnod.fesom.1953.nc \
-sellevidx,19 $datapath/vnod.fesom.1954.nc -sellevidx,19 $datapath/vnod.fesom.1955.nc \
-sellevidx,19 $datapath/vnod.fesom.1956.nc ] ] ] \
$datapath/mke_mean_95m_1951-1956.nc &

cdo timmean -mulc,0.5 -add [ -sqr [ -copy [ -sellevidx,19 $datapath/unod.fesom.2091.nc \
-sellevidx,19 $datapath/unod.fesom.2092.nc -sellevidx,19 $datapath/unod.fesom.2093.nc \
-sellevidx,19 $datapath/unod.fesom.2094.nc -sellevidx,19 $datapath/unod.fesom.2095.nc \
-sellevidx,19 $datapath/unod.fesom.2096.nc ] ] \
-sqr [ -copy [ -sellevidx,19 $datapath/vnod.fesom.2091.nc 
-sellevidx,19 $datapath/vnod.fesom.2092.nc -sellevidx,19 $datapath/vnod.fesom.2093.nc \
-sellevidx,19 $datapath/vnod.fesom.2094.nc -sellevidx,19 $datapath/vnod.fesom.2095.nc \
-sellevidx,19 $datapath/vnod.fesom.2096.nc ] ] ] \
$datapath/mke_mean_95m_2091-2096.nc &

In [ ]:
#execute
!bash mke_calcs.sh
!rm ./mke_calcs.sh

In [ ]:
#mke
mke50=xr.open_dataset(datapath+'mke_mean_95m_1951-1956.nc').isel(nz1=0,time=0).unod
mke90=xr.open_dataset(datapath+'mke_mean_95m_2091-2096.nc').isel(nz1=0,time=0).unod

In [ ]:
mke90_int=interp(mke90.values)
mke50_int=interp(mke50.values)

In [ ]:
#produce eke with cdo

In [ ]:
%%writefile eke_calcs.sh
module load cdo

datapath=/work/ab0995/a270166/freerange_runtime/fesom2.1/so3_from_init/so3-1950/chain

cdo ymonmean -copy [ -sellevidx,19 $datapath/unod.fesom.1951.nc -sellevidx,19 $datapath/unod.fesom.1952.nc \
-sellevidx,19 $datapath/unod.fesom.1953.nc -sellevidx,19 $datapath/unod.fesom.1954.nc \
-sellevidx,19 $datapath/unod.fesom.1955.nc -sellevidx,19 $datapath/unod.fesom.1956.nc ] ./unod_ymonmean_1951-1956_95m.nc

cdo ymonmean -copy [ -sellevidx,19 $datapath/vnod.fesom.1951.nc -sellevidx,19 $datapath/vnod.fesom.1952.nc \
-sellevidx,19 $datapath/vnod.fesom.1953.nc -sellevidx,19 $datapath/vnod.fesom.1954.nc \
-sellevidx,19 $datapath/vnod.fesom.1955.nc -sellevidx,19 $datapath/vnod.fesom.1956.nc ] ./vnod_ymonmean_1951-1956_95m.nc

cdo timmean -mulc,0.5 -add [ -sqr [ -ymonsub -copy [ -sellevidx,19 $datapath/unod.fesom.1951.nc \
-sellevidx,19 $datapath/unod.fesom.1952.nc -sellevidx,19 $datapath/unod.fesom.1953.nc \
-sellevidx,19 $datapath/unod.fesom.1954.nc -sellevidx,19 $datapath/unod.fesom.1955.nc \
-sellevidx,19 $datapath/unod.fesom.1956.nc ] ./unod_ymonmean_1951-1956_95m.nc ] \
-sqr [ -ymonsub -copy [ -sellevidx,19 $datapath/vnod.fesom.1951.nc \
-sellevidx,19 $datapath/vnod.fesom.1952.nc -sellevidx,19 $datapath/vnod.fesom.1953.nc \
-sellevidx,19 $datapath/vnod.fesom.1954.nc -sellevidx,19 $datapath/vnod.fesom.1955.nc \
-sellevidx,19 $datapath/vnod.fesom.1956.nc ] ./vnod_ymonmean_1951-1956_95m.nc ] ] \
./eke_mean_95m_1951-1956.nc

cdo ymonmean -copy [ -sellevidx,19 $datapath/unod.fesom.2091.nc -sellevidx,19 $datapath/unod.fesom.2092.nc \
-sellevidx,19 $datapath/unod.fesom.2093.nc -sellevidx,19 $datapath/unod.fesom.2094.nc \
-sellevidx,19 $datapath/unod.fesom.2095.nc -sellevidx,19 $datapath/unod.fesom.2096.nc ] ./unod_ymonmean_2091-2096_95m.nc

cdo ymonmean -copy [ -sellevidx,19 $datapath/vnod.fesom.2091.nc -sellevidx,19 $datapath/vnod.fesom.2092.nc \
-sellevidx,19 $datapath/vnod.fesom.2093.nc -sellevidx,19 $datapath/vnod.fesom.2094.nc \
-sellevidx,19 $datapath/vnod.fesom.2095.nc -sellevidx,19 $datapath/vnod.fesom.2096.nc ] ./vnod_ymonmean_2091-2096_95m.nc

cdo timmean -mulc,0.5 -add [ -sqr [ -ymonsub -copy [ -sellevidx,19 $datapath/unod.fesom.2091.nc \
-sellevidx,19 $datapath/unod.fesom.2092.nc -sellevidx,19 $datapath/unod.fesom.2093.nc \
-sellevidx,19 $datapath/unod.fesom.2094.nc -sellevidx,19 $datapath/unod.fesom.2095.nc \
-sellevidx,19 $datapath/unod.fesom.2096.nc ] ./unod_ymonmean_2091-2096_95m.nc ] \
-sqr [ -ymonsub -copy [ -sellevidx,19 $datapath/vnod.fesom.2091.nc \
-sellevidx,19 $datapath/vnod.fesom.2092.nc -sellevidx,19 $datapath/vnod.fesom.2093.nc \
-sellevidx,19 $datapath/vnod.fesom.2094.nc -sellevidx,19 $datapath/vnod.fesom.2095.nc \
-sellevidx,19 $datapath/vnod.fesom.2096.nc ] ./vnod_ymonmean_2091-2096_95m.nc ] ] \
./eke_mean_95m_2091-2096.nc

rm ./unod_ymonmean_1951-1956_95m.nc
rm ./vnod_ymonmean_1951-1956_95m.nc

In [ ]:
#execute
!bash eke_calcs.sh
!rm ./eke_calcs.sh

In [ ]:
#eke
eke50=xr.open_dataset(datapath+'eke_mean_95m_1951-1956.nc').isel(nz1=0,time=0).unod
eke90=xr.open_dataset(datapath+'eke_mean_95m_2091-2096.nc').isel(nz1=0,time=0).unod

In [ ]:
eke90_int=interp(eke90.values)
eke50_int=interp(eke50.values)

In [ ]:
def define_grid(gridtype='reg',bounds=[-180,180,-80,90],dx=1,dy=1,periodic=True):
    left,right,bottom,top=bounds
    #variables we want to keep
    global lons_c, lats_c, lons_gl, lats_gl, lons_gr, lats_gr, lats_g, \
    vpointslonl, vpointslatl, vpointslonr, vpointslatr, upointslonl, upointslatl, upointslonr, \
    upointslatr, cpointslon, cpointslat, fpointslonl, fpointslatl, fpointslonr, fpointslatr, \
    dxF, dyF, dxC, dyC, dxG, dyG, dxV, dyU 
    
    if gridtype=='reg':
        # the x center and f points on either side
        lons_c = np.arange(left+(0.5*dx), right, dx)
        lons_gl = np.arange(left, right, dx)
        lons_gr = lons_gl+(dx)
        
        lats_c = np.arange(bottom+(0.5*dy), top, dy)
        lats_gl = np.arange(bottom, top, dy)
        lats_gr = lats_gl+(dy)
        
    elif gridtype=='iso':
        
        # the x center and f points on either side
        lons_c = np.arange(left+(0.5*dx), right, dx)
        lons_gl = np.arange(left, right, dx)
        lons_gr = lons_gl+(dx)
        
        #the y 
        lats_g=[bottom]

        #we need one extra center lat point in order to differentiate the correct number of v points
        while lats_g[-1] < top+(dy*(np.cos(np.radians(top)))):
            lats_g.append(lats_g[-1]+dy*(np.cos(np.radians(lats_g[-1]))))

        lats_gl=lats_g[:-1]
        lats_gr=lats_g[1:]
        lats_c=(np.asarray(lats_gl)+np.asarray(lats_gr))/2

        lats_g,lats_gl,lats_gr=lats_g[:-1],lats_gl[:-1],lats_gr[:-1]
        
    else:
        print('grid type not prepared with this function')
        pass
        
    #latlon points in 2d
    vpointslonl, vpointslatl = np.meshgrid(lons_c,lats_gl)
    vpointslonr, vpointslatr = np.meshgrid(lons_c,lats_gr)

    upointslonl, upointslatl = np.meshgrid(lons_gl,lats_c)
    upointslonr, upointslatr = np.meshgrid(lons_gr,lats_c)

    cpointslon, cpointslat = np.meshgrid(lons_c,lats_c)

    fpointslonl, fpointslatl = np.meshgrid(lons_gl,lats_gl)
    fpointslonr, fpointslatr = np.meshgrid(lons_gr,lats_gr)
    
    #distances
    dxC = et.tracking.get_distance_latlon2m(cpointslon[:-1,1:],cpointslat[:-1,1:], 
                                            cpointslon[:-1,:-1],cpointslat[:-1,:-1])
    
    #we use the left lat point because we lose the top row 
    dxV = et.tracking.get_distance_latlon2m(vpointslonl[:,1:],vpointslatl[:,1:], 
                                            vpointslonl[:,:-1],vpointslatl[:,:-1])
    
    #for dxC and dxV we need to manually add the periodic distance
    if periodic:
        endcolumn=et.tracking.get_distance_latlon2m(cpointslon[:-1,0],cpointslat[:-1,0],
                                                    cpointslon[:-1,-1],cpointslat[:-1,-1])
        dxC = np.append(dxC, endcolumn.reshape((endcolumn.shape[0],1)),axis=1)
        endcolumn=et.tracking.get_distance_latlon2m(vpointslonl[:,0],vpointslatl[:,0],
                                                    vpointslonl[:,-1],vpointslatl[:,-1])
        dxV = np.append(dxV, endcolumn.reshape((endcolumn.shape[0],1)),axis=1)

    #for dyC and dyU, we lose the top row that we added above
    #*** to do: change to lose the bottom row, so that this works better in the northern hemisphere
    #the bottom row can be on land over antarctica
    dyC = et.tracking.get_distance_latlon2m(cpointslon[1:,:],cpointslat[1:,:],
                                            cpointslon[:-1,:],cpointslat[:-1,:])
    dyU = et.tracking.get_distance_latlon2m(upointslonl[1:,:],upointslatl[1:,:],
                                            upointslonl[:-1,:],upointslatl[:-1,:])

    #dxG and dyG
    dxG = et.tracking.get_distance_latlon2m(fpointslonr,fpointslatl,fpointslonl,fpointslatl)
    dyG = et.tracking.get_distance_latlon2m(fpointslonl,fpointslatr,fpointslonl,fpointslatl)

    #dxF and dyF. for dxF we have to manually remove the top layer
    dxF = et.tracking.get_distance_latlon2m(upointslonr,upointslatl,upointslonl,upointslatl)
    dxF = dxF[:-1,:]
    dyF = et.tracking.get_distance_latlon2m(vpointslonl,vpointslatr,vpointslonl,vpointslatl)
    
    lats_c=lats_c[:-1]
    cpointslat=cpointslat[:-1,:]
    upointslonl=upointslonl[:-1,:]
    upointslatl=upointslatl[:-1,:]
    upointslonr=upointslonr[:-1,:]
    upointslatr=upointslatr[:-1,:]
    
define_grid('iso',[-180,180,-80,-40],dx=0.05,dy=0.05, periodic=True)

areas=dyU*dxV

In [ ]:
#plotting

In [ ]:
#contourlabel format
def fmtx(x):
    s = f"{x:.1f}"
    if s.endswith("0"):
        s = f"{x:.0f}"
    return rf"{s} \Sv" if plt.rcParams["text.usetex"] else f"{s} Sv"

labels=['a','b','c','d','e','f','g']

In [ ]:
fig,axes=simple_plot(dpi=300,
                ygridlocs=[-80,-75,-70,-65,-60],
                figsize=(12,10),
                box=[-180,180,-80,-59],
                rows=2,
                cols=3)

#titles on first row
titlesize=20
axes[0,0].set_title('1951-1956',fontsize=titlesize)
axes[0,1].set_title('2091-2096',fontsize=titlesize)
axes[0,2].set_title('Change',fontsize=titlesize)

plt.rc('xtick',labelsize=14)
plt.rc('ytick',labelsize=14)
plt.rc('axes',labelsize=14)

#land mask
for ax in axes.flatten():
    ax.add_geometries(mesh.lsmask_p, crs=ccrs.PlateCarree(), 
                              facecolor=[0.6,0.6,0.6], edgecolor='none' ,linewidth=1,zorder=100)
for num,ax in enumerate(axes[:,0]):
    ax.annotate(labels[num], xy=(0.05, 0.9),xycoords='axes fraction',horizontalalignment='left', 
                     verticalalignment='bottom',fontsize=22,weight='bold')

#eke 
cax_0=axes[0,0].pcolormesh(xint,yint,np.log10(eke5nt_int*10000),vmin=-1,vmax=2,cmap=rm,transform=ccrs.PlateCarree())
axes[0,1].pcolormesh(xint,yint,np.log10(eke9nt_int*10000),vmin=-1,vmax=2,cmap=rm,transform=ccrs.PlateCarree())
dax_0=axes[0,2].pcolormesh(xint,yint,(eke90_int)*10000-(eke50_int)*10000,cmap=rbm,transform=ccrs.PlateCarree(),norm=colors.SymLogNorm(linthresh=0.1, linscale=0.01,vmin=-100, vmax=100, base=10),)

#row 0 colorbars
cbar=plt.colorbar(cax_0,ax=axes[0,:2],orientation='horizontal',label='Eddy kinetic energy ($cm^2$/$s^2$)',extend='max')
cbar.set_ticks(ticks=[-1,0,1,np.log10(100)],labels=['0.1','1','10','100'])
dbar=plt.colorbar(dax_0,ax=axes[0,2],orientation='horizontal',label='Δ Eddy kinetic energy ($cm^2$/$s^2$)',extend='both')
dbar.set_ticks(ticks=[-100,-10,-1,0,1,10,100],labels=['-100','-10','1','0','1','10','100'])

#MKE
cax_1=axes[1,0].pcolormesh(xint,yint,mke50_int*10000,vmin=0,vmax=30,cmap=rm,transform=ccrs.PlateCarree())
axes[1,1].pcolormesh(xint,yint,mke90_int*10000,vmin=0,vmax=30,cmap=rm,transform=ccrs.PlateCarree())
dax_1=axes[1,2].pcolormesh(xint,yint,(mke90_int*10000)-(mke50_int*10000),vmin=-50,vmax=50,cmap=rbm,transform=ccrs.PlateCarree())

#row 1 colorbars
plt.colorbar(cax_1,ax=axes[1,:2],orientation='horizontal',label='Mean kinetic energy ($cm^2$/$s^2$)')
plt.colorbar(dax_1,ax=axes[1,2],orientation='horizontal',label='Δ Mean kinetic energy ($cm^2$/$s^2$)')

plt.savefig(figpath+'figure_2.png',bbox_inches='tight')

In [ ]:
#Figure 4
fig,axes=simple_plot(dpi=300,
                ygridlocs=[-80,-75,-70,-65,-60],
                figsize=(12,15),
                box=[-180,180,-80,-59],
                rows=3,
                cols=3)

#titles on first row
titlesize=20
axes[0,0].set_title('1951-1956',fontsize=titlesize)
axes[0,1].set_title('2091-2096',fontsize=titlesize)
axes[0,2].set_title('Change',fontsize=titlesize)

plt.rc('xtick',labelsize=14)
plt.rc('ytick',labelsize=14)
plt.rc('axes',labelsize=14)

#land mask
for ax in axes.flatten():
    ax.add_geometries(mesh.lsmask_p, crs=ccrs.PlateCarree(), 
                              facecolor=[0.6,0.6,0.6], edgecolor='none' ,linewidth=1,zorder=100)
for num,ax in enumerate(axes[:,0]):
    ax.annotate(labels[num], xy=(0.05, 0.9),xycoords='axes fraction',horizontalalignment='left', 
                     verticalalignment='bottom',fontsize=22,weight='bold')

#streamfunction
#contourf plots
cax_0=axes[0,0].contourf(streamf50.nlon,streamf50.nlat,streamf50.hbstreamf,levels=np.linspace(120,210,10),cmap=rm,extend='both',transform=ccrs.PlateCarree())
axes[0,1].contourf(streamf90.nlon,streamf90.nlat,streamf90.hbstreamf,levels=np.linspace(120,210,10),cmap=rm,extend='both',transform=ccrs.PlateCarree())
dax_0=axes[0,2].contourf(streamf50.nlon,streamf50.nlat,streamf90.hbstreamf-streamf50.hbstreamf,levels=np.linspace(-10,10,9),cmap=rbm,extend='both',transform=ccrs.PlateCarree())

#contours
conax50_00=axes[0,0].contour(streamf50.nlon,streamf50.nlat,streamf50.hbstreamf,levels=np.linspace(120,210,10),colors='k',alpha=1,linewidths=0.7,transform=ccrs.PlateCarree())
conax90_01=axes[0,1].contour(streamf90.nlon,streamf90.nlat,streamf90.hbstreamf,levels=np.linspace(120,210,10),colors='k',alpha=1,linewidths=0.7,linestyles='--',transform=ccrs.PlateCarree())
conax50_02=axes[0,2].contour(streamf50.nlon,streamf50.nlat,streamf50.hbstreamf,levels=np.linspace(120,210,10),colors='k',alpha=1,linewidths=0.7,transform=ccrs.PlateCarree())
conax90_02=axes[0,2].contour(streamf90.nlon,streamf90.nlat,streamf90.hbstreamf,levels=np.linspace(120,210,10),colors='k',alpha=1,linewidths=0.7,linestyles='--',transform=ccrs.PlateCarree())

#row 0 colorbars
plt.colorbar(cax_0,ax=axes[0,:2],orientation='horizontal',label='Horizontal barotropic streamfunction (Sv)')
plt.colorbar(dax_0,ax=axes[0,2],orientation='horizontal',label='Δ Horizontal barotropic streamfunction (Sv)',ticks=[-10,-5,0,5,10])

#density
cax_1=axes[1,0].pcolormesh(dens50.lon,dens50.lat,dens50,vmin=1026.8,vmax=1028.2,cmap='Reds',transform=ccrs.PlateCarree())
axes[1,1].pcolormesh(dens50.lon,dens50.lat,dens90,vmin=1026.8,vmax=1028.2,cmap='Reds',transform=ccrs.PlateCarree())
dax_1=axes[1,2].pcolormesh(dens50.lon,dens50.lat,dens90-dens50,vmin=-0.5,vmax=0.5,cmap='RdBu_r',transform=ccrs.PlateCarree())

#row 1 colorbars
plt.colorbar(cax_1,ax=axes[1,:2],orientation='horizontal',label='Density (kg/$m^3$)',extend='both')
plt.colorbar(dax_1,ax=axes[1,2],orientation='horizontal',label='Δ Density (kg/$m^3$)',extend='both')

#Zonal winds
cax_2=axes[2,0].contourf(uw50.lon,uw50.lat,uw50.uas,levels=np.linspace(-10,10,41),cmap=rbm,extend='both',transform=ccrs.PlateCarree())
axes[2,1].contourf(uw90.lon,uw90.lat,uw90.uas,levels=np.linspace(-10,10,41),cmap=rbm,extend='both',transform=ccrs.PlateCarree())
dax_2=axes[2,2].contourf(uw50.lon,uw50.lat,uw90.uas-uw50.uas,levels=np.linspace(-3,3,41),cmap=rbm,extend='both',transform=ccrs.PlateCarree())

#row 2 colorbars
plt.colorbar(cax_2,ax=axes[2,:2],orientation='horizontal',label='Zonal surface wind speed (m/s Eastward)')
plt.colorbar(dax_2,ax=axes[2,2],orientation='horizontal',label='Δ Zonal surface wind speed (m/s Eastward)',ticks=[-3,-1.5,0,1.5,3])

plt.savefig(figpath+'figure_4.png',bbox_inches='tight')